# Cost Matrix or Waypoint Graph

### Introduction
The cost matrix and the waypoint graph are two environment representations for the NVIDIA cuOpt Self-hosted Service. Each one encapsulates user-defined costs of transitioning from one state or location to another within an optimization problem. cuOpt uses this environment representation to evaluate solution quality while minimizing the total cost.

### Deciding Between a Cost Matrix and a Waypoint Graph
The selection hinges on your data availability and use case specifics.

- **Cost Matrices :** Choose a cost matrix when your primary focus is the cost between target locations, such as orders, jobs, or vehicles. This method is common in applications reliant on map data, where cost values between required points is either known or easily calculated.

- **Waypoint Graph :** Choose a waypoint graph when your data is portrayed as a network where a subset of nodes represent target locations and others are only being traversed (not acted upon by vehicles or agents, and lack associated orders or jobs). This approach is common in custom environments and indoor locations like warehouses and factories where the cost between target locations is dynamic or not easily calculated.

In [ ]:
import subprocess
import html
from IPython.display import display, HTML

def check_gpu():
    try:
        result = subprocess.run(["nvidia-smi"], capture_output=True, text=True, timeout=5)
        result.check_returncode()
        lines = result.stdout.splitlines()
        gpu_info = lines[2] if len(lines) > 2 else "GPU detected"
        gpu_info_escaped = html.escape(gpu_info)
        display(HTML(f"""
        <div style="border:2px solid #4CAF50;padding:10px;border-radius:10px;background:#e8f5e9;">
            <h3>✅ GPU is enabled</h3>
            <pre>{gpu_info_escaped}</pre>
        </div>
        """))
        return True
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired, FileNotFoundError, IndexError) as e:
        display(HTML("""
        <div style="border:2px solid red;padding:15px;border-radius:10px;background:#ffeeee;">
            <h3>⚠️ GPU not detected!</h3>
            <p>This notebook requires a <b>GPU runtime</b>.</p>
            
            <h4>If running in Google Colab:</h4>
            <ol>
              <li>Click on <b>Runtime → Change runtime type</b></li>
              <li>Set <b>Hardware accelerator</b> to <b>GPU</b></li>
              <li>Then click <b>Save</b> and <b>Runtime → Restart runtime</b>.</li>
            </ol>
            
            <h4>If running in Docker:</h4>
            <ol>
              <li>Ensure you have <b>NVIDIA Docker runtime</b> installed (<code>nvidia-docker2</code>)</li>
              <li>Run container with GPU support: <code>docker run --gpus all ...</code></li>
              <li>Or use: <code>docker run --runtime=nvidia ...</code> for older Docker versions</li>
              <li>Verify GPU access: <code>docker run --gpus all nvidia/cuda:12.0.0-base-ubuntu22.04 nvidia-smi</code></li>
            </ol>
            
            <p><b>Additional resources:</b></p>
            <ul>
              <li><a href="https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html" target="_blank">NVIDIA Container Toolkit Installation Guide</a></li>
            </ul>
        </div>
        """))
        return False

check_gpu()

In [ ]:
# Install notebook dependencies
!pip install -q scipy matplotlib pandas requests polyline folium

In [12]:
from scipy.spatial import distance
import numpy as np
import pandas as pd
import requests
import polyline
import folium
import json

# Cost Matrix

A cost matrix is a two-dimensional array, where each cell represents the cost or distance between two points. The matrix's rows and columns correspond to different nodes in the routing problem, with each entry showing the cost of transitioning from one point to another. The "cost" can encapsulate a range of metrics, from monetary values to distance, time, or energy. This matrix serves an important function allowing cuOpt to compute the optimal route (minimize the total cost).

<img src="cost_matrix.png" alt="cost_matrix.png not found]" width="750"/>

**Additional Notes:**
- The cost of going from a location to itself (Cost(A,A)) is typically 0 
- Cost(A,B) need not be equal to Cost(B,A).  Asymmetric matrices are allowed.

### Simple Cost Matrix

In some simple cases a cost matrix can be computed directly via a user defined metric (e.g. Euclidean, Manhattan, or something more complicated).  Here we show a simple example of cost matrix generation leveraging Euclidean distance from a point list.

In [ ]:
points = pd.DataFrame({"x_coord": [1, 1, 2, 3], "y_coord":[3, 1, 4, 1]})
points

In [ ]:
cost_matrix = distance.cdist(points.values, points.values, "euclidean")
print(f"Simple Metric Cost Matrix:\n\n{cost_matrix}")

### Map Based Cost Matrix

When dealing with problems in shipping and logistics, road distance and/or time is often used as a cost metric.  In these cases there are a number of tools available to calculate drive distance and/or time.  One such tool is the [Open Source Routing Machine](http://project-osrm.org/)(OSRM).  In the below example we create a cost matrix using OSRM from a list of lat/lon coordinates.

#### Define Points of Interest

In [5]:
lat_lon_coords = [
    [48.137467, 11.622837],
    [48.137626, 11.601777], 
    [48.150541, 11.618471], 
    [48.131611, 11.638083]
] 

#### Create Distance Matrix via OSRM

In [ ]:
locations=""
for loc in lat_lon_coords:
    locations = locations + "{},{};".format(loc[1], loc[0])
r = requests.get("http://router.project-osrm.org/table/v1/driving/"+ locations[:-1])

routes = json.loads(r.content)
cols = [str(i) for i in lat_lon_coords]
cost_matrix = pd.DataFrame(routes['durations'], columns = cols, index= cols)
print(f"Cost Matrix via OSRM:\n")
cost_matrix

#### Map Visualization

Visualization can be a helpful tool for understanding and communication. Here we demonstrate one method for visualizing (map based) locations and the path between them.  This example shows all paths dictated by the above cost matrix.  A similar approach can be taken to show the optimized routes generated by cuOpt.

In [ ]:
def get_map(my_lat_longs):
    m = folium.Map(location=[48.137467, 11.622837],
                   zoom_start=13)
    folium.Marker(
        location=[my_lat_longs[0][0],my_lat_longs[0][1]] ,
        icon=folium.Icon(icon='play', color='red')
    ).add_to(m)
    for loc in my_lat_longs[1:]:
        folium.Marker(
            location=[loc[0], loc[1]],
            icon=folium.Icon(icon='stop', color='green')
        ).add_to(m)
            
    for src_idx in range(len(lat_lon_coords)):
        for dst_idx in range(len(lat_lon_coords)):
            if src_idx == dst_idx:
                break
            source = lat_lon_coords[src_idx]
            destination = lat_lon_coords[dst_idx]
            loc = "{},{};{},{}".format(source[1], source[0], destination[1], destination[0])
            url = "http://router.project-osrm.org/route/v1/driving/"
            r = requests.get(url + loc) 

            res = r.json()   
            routes = polyline.decode(res['routes'][0]['geometry'])

            folium.PolyLine(
                routes,
                weight=5,
                color='blue',
                opacity=0.6
            ).add_to(m)

    return m
get_map(lat_lon_coords)

# Waypoint Graph

<img src="waypoint_graph.png" alt="waypoint_graph.png not found]" width="550"/>

#### Graph Conversion
In the example below we demonstrate the conversion from a typical graph representation to the required CSR format.

In [8]:
graph = {
    0:{
        "edges":[2], 
        "weights":[2]},
    1:{
        "edges":[2, 4], 
        "weights":[2, 2]},
    2:{
        "edges":[0, 1, 3, 5], 
        "weights":[2, 2, 2, 2]},
    3:{
        "edges":[2, 6], 
        "weights":[2, 2]},
    4:{
        "edges":[1, 7], 
        "weights":[2, 1]},
    5:{
        "edges":[2, 8], 
        "weights":[2, 1]},
    6:{
        "edges":[3, 9], 
        "weights":[2, 1]},
    7:{
        "edges":[4, 8], 
        "weights":[1, 2]},
    8:{
        "edges":[5, 7, 9], 
        "weights":[1, 2, 2]},
    9:{
        "edges":[6, 8], 
        "weights":[1, 2]}
}

#### Convert to CSR
Here we define a simple function that converts our graph to CSR.

In [9]:
def convert_to_csr(graph):
    num_nodes = len(graph)
    
    offsets = []
    edges = []
    weights = []
    
    cur_offset = 0
    for node in range(num_nodes):
        offsets.append(cur_offset)
        cur_offset += len(graph[node]["edges"])
        
        edges = edges + graph[node]["edges"]
        weights = weights + graph[node]["weights"]
        
    offsets.append(cur_offset)
    
    return np.array(offsets), np.array(edges), np.array(weights)

In [ ]:
offsets, edges, weights = convert_to_csr(graph)
print(f"offsets = {list(offsets)}")
print(f"edges   = {list(edges)}")
print(f"weights = {list(weights)}")

# Using the Environment Descirption

Whether you are employing a cost matrix or a waypoint graph in your use case, the environment description will be included in the JSON payload that is passed to the self hosted service. For an example implementation of each approach, please refer to the other example notebooks in the repository.


SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.